## Data Reading

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=spark.read.format('parquet').load('abfss://bronze2026@storagespotifyproject20.dfs.core.windows.net/orders')

## Drop column

In [0]:
df = df.drop("_rescued_data")

## Casting the order_date

In [0]:
df = df.withColumn("order_date", to_timestamp("order_date"))

## Adding New Column

In [0]:
df = df.withColumn("Year", year("order_date"))

In [0]:
import sys
import importlib.util

# Add the directory to sys.path
# sys.path.insert(0, '/Workspace/Databricks ETL Project')

# Load the module using importlib
spec = importlib.util.spec_from_file_location(
    "resuable_function", "/Workspace/Databricks ETL Project/resuable_function.py"
)
resuable_function = importlib.util.module_from_spec(spec)
spec.loader.exec_module(resuable_function)

# Import the class
window_function = resuable_function.window_function

## Object creation for class

In [0]:
df_orders_obj = window_function()

## Adding window function

In [0]:
# df=df_orders_obj.rank(df)
# df=df_orders_obj.dense_rank(df)
# df=df_orders_obj.row_num(df)

## Data Writing

In [0]:
df.write.format("delta").mode("append").save(
    "abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/orders"
)

## Create table in silver layer

In [0]:
%sql
create table if not exists databricks_cata.silver.Orders_silver
using delta
location 'abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/orders'